In [1]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
import numpy as np
import pandas as pd
import torch

In [2]:
MODEL_NAME = "512-encoder-1000K/kaggle/working/results/checkpoint-125000/"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(64101, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64101, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [3]:
model.generation_config
start_token = "<unk>"
end_token = "</s>"

In [4]:
from datasets import load_dataset

dataset = load_dataset("mateiaassAI/MEID_v2", split=['train[:97%]', 'train[97%:100%]'])
dataset

[Dataset({
     features: ['wrong', 'right'],
     num_rows: 1967695
 }),
 Dataset({
     features: ['wrong', 'right'],
     num_rows: 60857
 })]

In [5]:
ds_train = dataset[0]
ds_test = dataset[1]
ds_test

Dataset({
    features: ['wrong', 'right'],
    num_rows: 60857
})

In [6]:
punctuation_marks = ['.', '?', '!', ';', '...']

def filter_sentences(sentences):
    text = sentences['right']
    if any(text.endswith(punc) for punc in punctuation_marks):
      words = text.split()
      if len(words) >= 10:
        return True
    return False

In [7]:
fds_test = ds_test.filter(filter_sentences, batched=False)

In [8]:
fds_test[0]
print(len(fds_test))

31457


In [ ]:
import json

def extract_text(generated_text, start_token="<unk>", end_token="</s>"):
    start_index = generated_text.find(start_token)
    end_index = generated_text.find(end_token, start_index)
    if start_index != -1 and end_index != -1:
        return generated_text[start_index + len(start_token):end_index].strip()
    else:
        return None

sentence_pairs = []
batch_index = 10

for idx, data in enumerate(fds_test):
    if idx < 10000:
        continue;
        
    print(idx)
    correct_sentence = data['right']
    input_sentence = data['wrong']

    input_tensor = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
    outputs = model.generate(input_tensor)
    generated_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=False)
    predicted_sentence = extract_text(generated_text)

    if predicted_sentence is not None:
        sentence_pairs.append(
            (correct_sentence, input_sentence, predicted_sentence))

    if len(sentence_pairs) >= 1000:
        df = pd.DataFrame(sentence_pairs)
        file_name = f'ress/sentence_pairs_batch_{batch_index}.jsonl'
        with open(file_name, 'w', encoding='utf-8') as f:
            for pair in sentence_pairs:
                json.dump(pair, f, ensure_ascii=False)
                f.write('\n')
        print(f"Saved 1000 sentence pairs to {file_name}")
        sentence_pairs = []  # Clear the list after saving
        batch_index += 1  # Increment the batch index

    if batch_index == 15:
        break

10000
10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
1016

In [25]:
print(sentence_pairs[0][2])

Pentru perioada în care un pacient are deschisă o fișă de spitalizare de zi pe<unk> parcursul unei singure zile sau pe<unk> parcursul mai multor zile, acesta poate beneficia și de servicii medicale în ambulatoriul de specialitate, altele decât cele necesare acordării serviciilor medicale din spitalizare de zi, cu respectarea condițiilor de acordare a serviciilor medicale în ambulatoriu.


In [26]:
input_ids = tokenizer("Maria sa dus la mare ca sa mannce incchetata cu vanilie ", return_tensors="pt").input_ids
generated_text = tokenizer.decode(model.generate(input_ids)[0], skip_special_tokens=False)

start_index = generated_text.find(start_token)
end_index = generated_text.find(end_token, start_index)

if start_index != -1 and end_index != -1:
    extracted_text = generated_text[start_index + len(start_token):end_index].strip()
else:
    extracted_text = None

print(extracted_text)

Maria sa dus la mare ca sa manance incicheta cu vanilie.
